In [ ]:
import os
from datetime import UTC, datetime, timedelta

from pyspark.sql import SparkSession

os.environ["AWS_PROFILE"] = "blueriver"

CATALOG = "glue_catalog"
ICEBERG_S3_ROOT_PATH = "s3a://blueriver-datalake/iceberg"

spark = (
    SparkSession.builder.appName("3")
    .config("spark.sql.defaultCatalog", CATALOG)
    .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
    .config(f"spark.sql.catalog.{CATALOG}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config(f"spark.sql.catalog.{CATALOG}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config(f"spark.sql.catalog.{CATALOG}.warehouse", ICEBERG_S3_ROOT_PATH)
    .config(f"spark.sql.catalog.{CATALOG}.s3.path-style-access", True)
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "software.amazon.awssdk.auth.credentials.DefaultCredentialsProvider",
    )
    .config("spark.sql.caseSensitive", True)
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)

In [ ]:
# Note. 테이블 컬럼 정보
# DESCRIBE <schema>.<table>;
spark.sql("SHOW CREATE TABLE store_bronze.tb_lower;").show(truncate=False)

In [ ]:
# Note. 테이블 속성 정보
# SHOW TBLPROPERTIES [db_name.]table_name [('property_name')]
spark.sql("SHOW TBLPROPERTIES store_bronze.tb_lower;").show(truncate=False)

In [ ]:
# Note. 테이블 속성 추가
# ALTER TABLE [db_name.]table_name SET TBLPROPERTIES ('property_name' = 'property_value' [ , ... ])
spark.sql("ALTER TABLE store_bronze.tb_lower SET TBLPROPERTIES (''='');").show(truncate=False)

In [ ]:
# Note. 테이블 속성 제거
# ALTER TABLE [db_name.]table_name UNSET TBLPROPERTIES ('property_name' [ , ... ])
spark.sql(
    "ALTER TABLE store_bronze.tb_lower SET TBLPROPERTIES ('write.metadata.delete-after-commit.enabled' = 'true', 'write.metadata.previous-versions-max' = '1');"
).show(truncate=False)

In [ ]:
# Note. 테이블 이름 변경
# ALTER TABLE [db_name.]table_name RENAME TO [new_db_name.]new_table_name
spark.sql("ALTER TABLE store_bronze.tb_lower RENAME TO store_bronze.tb_lower;").show(truncate=False)

In [ ]:
# Note. 테이블 목록
# SHOW TABLES [IN database_name] ['regular_expression']
spark.sql("SHOW TABLES IN store_bronze;").show(truncate=False)

In [ ]:
# Note. 뷰 목록
# SHOW VIEWS [IN database_name] [LIKE 'regular_expression']
spark.sql("SHOW VIEWS IN store_bronze;").show(truncate=False)

In [ ]:
# Note. 테이블 파티션 정보
# SHOW PARTITIONS table_name
spark.sql("SHOW PARTITIONS store_bronze.tb_lower").show(truncate=False)

In [ ]:
# Note. 테이블 히스토리 정보
# SELECT * FROM "database_name"."table_name$history"
spark.sql("SELECT * FROM store_bronze.tb_lower.history").show(truncate=False)

In [ ]:
# Note. 테이블 스냅샷 정보
# SELECT * FROM "database_name"."table_name$snapshots"
spark.sql("SELECT * FROM store_bronze.tb_lower.snapshots").show(truncate=False)

In [ ]:
# Note. 테이블 매니페스트 정보
# SELECT * FROM "database_name"."table_name$manifests"
spark.sql("SELECT * FROM store_bronze.tb_lower.manifests").show(truncate=False)

In [ ]:
# Note. 스냅샷 만료
df = spark.sql("SELECT current_timestamp() AS current_time")
older_than = str(df.toPandas()["current_time"][0])
print(older_than)
spark.sql(
    f"CALL {CATALOG}.system.expire_snapshots(table => 'store_bronze.tb_lower', older_than	=> TIMESTAMP '{older_than}')"
).show(truncate=False)

In [ ]:
# Note. 고아 파일 삭제
older_than = datetime.now(UTC) - timedelta(days=1)
older_than = older_than.strftime("%Y-%m-%d %H:%M:%S")
print(older_than)
spark.sql(
    f"CALL {CATALOG}.system.remove_orphan_files(table => 'store_bronze.tb_lower', older_than => TIMESTAMP '{older_than}')"
).show(truncate=False)

In [ ]:
# Note. 매니페스트 쓰기
spark.sql(f"CALL {CATALOG}.system.rewrite_manifests(table => 'store_bronze.tb_lower')").show(truncate=False)

In [ ]:
# Note. 데이터 쓰기
spark.sql(
    f"CALL {CATALOG}.system.rewrite_data_files(table => 'store_bronze.tb_lower', options => map('min-input-files', '1'))"
).show(truncate=False)

In [ ]:
# Note. 포지션 삭제 파일 쓰기
spark.sql(f"CALL {CATALOG}.system.rewrite_position_delete_files(table => 'store_bronze.tb_lower')").show(truncate=False)